In [ ]:
%%html
<div style="text-align: center; background: #1a1a1a; padding: 20px; border-radius: 15px; color: white; font-family: sans-serif;">
    <h3>Método Sobel (Gradientes de Intensidad)</h3>
    <video id="v_sobel_pro" width="640" height="480" style="display:none" playsinline></video>
    <canvas id="c_sobel_pro" width="640" height="480" style="background: #000; border: 2px solid #2980b9; border-radius: 10px;"></canvas>
    <div style="margin-top: 15px;">
        <button id="btn_on_sobel" onclick="iniciarSobelPro()" style="padding: 10px 20px; background: #2980b9; color: white; border: none; border-radius: 5px; cursor: pointer; font-weight: bold;">🚀 ACTIVAR SOBEL</button>
        <button id="btn_off_sobel" onclick="detenerTodo()" style="padding: 10px 20px; background: #e74c3c; color: white; border: none; border-radius: 5px; cursor: pointer; margin-left: 10px; display: none;">🛑 APAGAR</button>
    </div>
    <p id="log_sobel_pro" style="color: #f1c40f; font-size: 13px; margin-top: 10px;">Estado: Esperando cámara...</p>
</div>

<script>
// Implementación de Sobel
function applySobelGradients(imageData) {
    const width = imageData.width;
    const height = imageData.height;
    const data = imageData.data;
    
    // Convertir a escala de grises
    const gray = new Uint8ClampedArray(width * height);
    for (let i = 0; i < data.length; i += 4) {
        const idx = i / 4;
        gray[idx] = 0.299 * data[i] + 0.587 * data[i + 1] + 0.114 * data[i + 2];
    }
    
    // Aplicar Sobel
    const gradX = new Float32Array(width * height);
    const gradY = new Float32Array(width * height);
    const magnitude = new Uint8ClampedArray(width * height);
    
    for (let y = 1; y < height - 1; y++) {
        for (let x = 1; x < width - 1; x++) {
            const idx = y * width + x;
            
            // Sobel X (detecta bordes verticales)
            const gx = (
                -gray[(y-1)*width + (x-1)] + gray[(y-1)*width + (x+1)] +
                -2*gray[y*width + (x-1)] + 2*gray[y*width + (x+1)] +
                -gray[(y+1)*width + (x-1)] + gray[(y+1)*width + (x+1)]
            );
            
            // Sobel Y (detecta bordes horizontales)
            const gy = (
                -gray[(y-1)*width + (x-1)] - 2*gray[(y-1)*width + x] - gray[(y-1)*width + (x+1)] +
                gray[(y+1)*width + (x-1)] + 2*gray[(y+1)*width + x] + gray[(y+1)*width + (x+1)]
            );
            
            gradX[idx] = gx;
            gradY[idx] = gy;
            
            // Magnitud del gradiente
            const mag = Math.sqrt(gx * gx + gy * gy);
            magnitude[idx] = Math.min(255, mag);
        }
    }
    
    // Convertir a ImageData
    const output = new ImageData(width, height);
    for (let i = 0; i < magnitude.length; i++) {
        output.data[i * 4] = magnitude[i];
        output.data[i * 4 + 1] = magnitude[i];
        output.data[i * 4 + 2] = magnitude[i];
        output.data[i * 4 + 3] = 255;
    }
    
    return output;
}

async function iniciarSobelPro() {
    const log = document.getElementById('log_sobel_pro');
    
    // Detener otros filtros Y limpiar sus canvas
    detenerTodo();
    
    // IMPORTANTE: Limpiar TODOS los canvas antes de empezar
    ['c_canny_pro', 'c_out', 'c_sobel_pro'].forEach(canvasId => {
        const canvas = document.getElementById(canvasId);
        if (canvas) {
            const ctx = canvas.getContext('2d');
            ctx.clearRect(0, 0, canvas.width, canvas.height);
            ctx.fillStyle = '#000';
            ctx.fillRect(0, 0, canvas.width, canvas.height);
        }
    });
    
    try {
        log.innerText = "📷 Solicitando cámara...";
        
        const stream = await navigator.mediaDevices.getUserMedia({ 
            video: { width: 640, height: 480, facingMode: 'user' } 
        });
        
        window.cameraManager.currentStream = stream;
        window.cameraManager.currentFilter = 'sobel';
        
        const v = document.getElementById('v_sobel_pro');
        const canvas = document.getElementById('c_sobel_pro');
        const ctx = canvas.getContext('2d');
        
        v.srcObject = stream;
        await v.play();
        
        await new Promise(resolve => setTimeout(resolve, 500));
        
        document.getElementById('btn_on_sobel').style.display = 'none';
        document.getElementById('btn_off_sobel').style.display = 'inline-block';
        log.innerText = "✅ Gradientes Sobel activos";
        
        window.cameraManager.activeLoop = true;
        let frameCount = 0;
        
        function processFrame() {
            if (!window.cameraManager.activeLoop || window.cameraManager.currentFilter !== 'sobel') {
                return;
            }
            
            try {
                ctx.drawImage(v, 0, 0, canvas.width, canvas.height);
                const imageData = ctx.getImageData(0, 0, canvas.width, canvas.height);
                
                // Aplicar SOBEL
                const edges = applySobelGradients(imageData);
                
                ctx.putImageData(edges, 0, 0);
                
                frameCount++;
                if (frameCount % 30 === 0) {
                    log.innerText = "✅ Sobel activo - Frames: " + frameCount;
                }
                
                requestAnimationFrame(processFrame);
                
            } catch (e) {
                console.error('Error:', e);
                log.innerText = "❌ Error: " + e.message;
            }
        }
        
        processFrame();
        
    } catch (e) {
        log.innerText = "❌ Error: " + e.message;
        console.error(e);
    }
}
</script>